# Mnist with TensorFlow 

In [2]:
# Allow auto-reload of external modules 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Fully-connected Feedforward Neural Network

In [27]:
import NeuralNetwork.TensorFlow.simple_nn as nn
hidden = 300
w1,w2,loss, logit = nn.getTrainedWeight(hidden)

Final Eval:
100 10000
0.9033
  Num examples: 10000  Num correct: 9033  Precision @ 1: 0.9033
Step 1999: loss = 0.41


In [32]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os.path
import time
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import numpy as np

#from tensorflow.examples.tutorials.mnist import input_data
import NeuralNetwork.TensorFlow.input_data as input_data
import NeuralNetwork.TensorFlow.mnist as mst
# Arguments Definition

learning_rate = 0.01
max_steps = 2000
hidden1 = 300
hidden2 = 32
batch_size = 100
fake_data = False
train_dir = 'data'

# Load data_set
data_sets = input_data.read_data_sets(train_dir, fake_data)



                 
def placeholder_inputs(batch_size):
  images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                         mst.IMAGE_PIXELS))
  labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
  return images_placeholder, labels_placeholder

def fill_feed_dict(data_set, batch_size, images_pl, labels_pl):
  images_feed, labels_feed = data_set.next_batch(batch_size,fake_data)
  feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
  }
  return feed_dict


def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_set):
  """Runs one evaluation against the full epoch of data.

  Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
  """
  # And run one epoch of eval.
  true_count = 0  # Counts the number of correct predictions.
  steps_per_epoch = data_set.num_examples // batch_size
  num_examples = steps_per_epoch * batch_size
  print(steps_per_epoch, num_examples)
  for step in xrange(steps_per_epoch):
    feed_dict = fill_feed_dict(data_set, batch_size,
                               images_placeholder,
                               labels_placeholder)
    count = sess.run(eval_correct, feed_dict=feed_dict)
    true_count += count 
  precision = true_count / num_examples
  print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))


    
# Tell TensorFlow that the model will be built into the default Graph.
with tf.Graph().as_default() as g:
    # Generate placeholders for the images and labels.
    images_placeholder, labels_placeholder = placeholder_inputs(batch_size)

    # Build a Graph that computes predictions from the inference model.
    logits = mst.inference2(images_placeholder,hidden1)
    #logits = mst.inference(images_placeholder,hidden1,hidden2)
    # Add to the Graph the Ops for loss calculation.
    loss = mst.loss(logits, labels_placeholder)

    # Add to the Graph the Ops that calculate and apply gradients.
    train_op = mst.training(loss, learning_rate)

    # Add the Op to compare the logits to the labels during evaluation.
    eval_correct = mst.evaluation(logits, labels_placeholder)
    init = tf.initialize_all_variables()

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Create a session for running Ops on the Graph.
    sess = tf.Session()
    sess.run(init)
    
    # Start the training loop.
    for step in xrange(max_steps):
        start_time = time.time()
        feed_dict = fill_feed_dict(data_sets.train, batch_size, images_placeholder, labels_placeholder)
        _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)
        
        duration = time.time() - start_time       
        # Write the summaries and print an overview fairly often.
        if step % 200 == 0:
        # Print status to stdout.
            print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))

        # Save a checkpoint and evaluate the model periodically.
#         if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
#             print('Print logits:')
#             print('Training Data Eval:')
#             do_eval(sess,
#                     eval_correct,
#                     images_placeholder,
#                     labels_placeholder,
#                     data_sets.train)
        
#             # Evaluate against the validation set.
#             print('Validation Data Eval:')
#             do_eval(sess,
#                     eval_correct,
#                     images_placeholder,
#                     labels_placeholder,
#                     data_sets.validation)
#             # Evaluate against the test set.
#             print('Test Data Eval:')
#             do_eval(sess,
#                     eval_correct,
#                     images_placeholder,
#                     labels_placeholder,
#                     data_sets.test)
            
    print('Final Eval:')
    do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_sets.test)
    print('Step %d: loss = %.2f' % (step, loss_value))    
    
    w1 = g.get_collection(tf.GraphKeys.VARIABLES, "hidden1/weights")
    w2 = g.get_collection(tf.GraphKeys.VARIABLES, "softmax_linear/weights")
    w1_val,w2_val = sess.run([w1,w2])
#     print(w1_val,w2_val)



Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Step 0: loss = 2.27 (0.022 sec)
Step 200: loss = 1.36 (0.005 sec)
Step 400: loss = 1.00 (0.005 sec)
SHUFFLE
Step 600: loss = 0.70 (0.005 sec)
Step 800: loss = 0.47 (0.005 sec)
Step 1000: loss = 0.47 (0.005 sec)
SHUFFLE
Step 1200: loss = 0.41 (0.005 sec)
Step 1400: loss = 0.56 (0.005 sec)
Step 1600: loss = 0.33 (0.005 sec)
SHUFFLE
Step 1800: loss = 0.45 (0.005 sec)
Final Eval:
100 10000
  Num examples: 10000  Num correct: 8996  Precision @ 1: 0.8996
Step 1999: loss = 0.38


In [31]:
w1_v = sess.run(w1)
print(w1_v[0].shape)

w2_v = sess.run(w2)
print(w2_v[0].shape)

(784, 300)
(300, 10)


Problem:
1. w is in size of 100: saver to record all of them then concat them together or adapt neuralnetwork.py
2. how to check whether it gains better loss, if we don't have z, or let z = wa? is it not accurate because in our network it's not the case.
3. mini-batch is much much much faster than normal.

In [20]:
log = sess.run(g.get_collection(tf.GraphKeys.VARIABLES, "softmax_linear/weights"))
print(step,max_steps)

1999 2000


In [19]:
w2_val

[array([[ 0.08991498, -0.04007696,  0.03823632, ..., -0.04408926,
         -0.06910585,  0.05061473],
        [-0.11108518, -0.01257499,  0.11620861, ...,  0.08067908,
          0.06203641, -0.09924033],
        [-0.05920425, -0.04707693,  0.01908229, ..., -0.03129459,
         -0.05405887, -0.04370198],
        ..., 
        [-0.14072941,  0.0451174 , -0.0910515 , ...,  0.13482447,
         -0.05188115,  0.13384832],
        [ 0.0023597 ,  0.09977771,  0.01873406, ..., -0.05493696,
         -0.09123113,  0.06701973],
        [-0.041249  , -0.03007073, -0.0700364 , ..., -0.1045787 ,
         -0.02126557, -0.05199299]], dtype=float32)]

In [7]:
np.set_printoptions(precision=3)
w = W.eval()
[x,y]=np.where( w > 0.0 )
for i in range(10):
    print w[x[i],y[i]]


0.000441887
0.00327859
0.00306615
0.00108164
0.00383746
0.00617507
0.00598686
0.00462077
0.00205038
0.000815012


In [8]:
print w.shape

(784, 10)


# Simple Layer Classifier

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

# Initial input img and output label
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# Train network
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.initialize_all_variables())
y = tf.matmul(x,W) #+ b
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Mini batch descent
for i in range(100):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# Predict
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.8902
